In [1]:
from sklearn.impute import SimpleImputer, KNNImputer
import os
from src.dataPipeline import DataPipeline


os.chdir('..')

imputer = KNNImputer(n_neighbors=5)
dp = DataPipeline()
df = dp.runPipeline(normalizeAndStandardize=False, imputer=imputer)

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:39: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To o

In [2]:
inputs_nums = len(df.columns) - 1
print(f'input nums: {inputs_nums}')

input nums: 81


In [4]:
from src import torchModelRun
import torch
import torch.nn as nn
import torch.nn.functional as F

inputs_nums = len(df.columns) - 1

class FullyConnectedModel(nn.Module):
    import torch.nn.functional as F
    def __init__(self):
        super(FullyConnectedModel, self).__init__()
        self.fc1 = nn.Linear(inputs_nums, 76)
        self.fc2 = nn.Linear(76, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    

models = torchModelRun.run(FullyConnectedModel, df, 5)

wandb: Currently logged in as: denis-schatzmann. Use `wandb login --relogin` to force relogin


Epoch 1, Loss: 71.48512347535456, Train MAPE: 228.30810715848168, Test MAPE: 229.85431636064902
Epoch 2, Loss: 0.9901591422230949, Train MAPE: 76.12116872909999, Test MAPE: 76.82599197036919
Epoch 3, Loss: 0.540850488902967, Train MAPE: 59.35464150199268, Test MAPE: 59.98922987459263
Epoch 4, Loss: 0.4410339267790489, Train MAPE: 56.40339634128179, Test MAPE: 57.27292271318107
Epoch 5, Loss: 0.3856259947348018, Train MAPE: 51.69134362111956, Test MAPE: 52.66603867395627
Epoch 6, Loss: 0.3478740379428589, Train MAPE: 49.789888305152026, Test MAPE: 50.734631629739226
Epoch 7, Loss: 0.31976177349100865, Train MAPE: 46.19609725418164, Test MAPE: 46.98389296696104
Epoch 8, Loss: 0.29723573936111974, Train MAPE: 44.59978915769553, Test MAPE: 45.37005914856191
Epoch 9, Loss: 0.2791165662134711, Train MAPE: 42.51996424739907, Test MAPE: 43.218195999262434
Epoch 10, Loss: 0.2639170162136866, Train MAPE: 40.19170146020467, Test MAPE: 40.8277087010643
Epoch 11, Loss: 0.2511614859282742, Train MAP

test_loss,█▅▄▄▄▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_mape,█▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_test,▇▅▄▅▂▂▁▃▂▂▃▂▂▃▂▂▃▂▁▃▂█▃▂▃▂▂▂▄▂▂▃▆▄▃▄▁▆▂▂
time_train,▃▄█▆▃▂▂▅▁▂▂▃▂▂▂▃▂▂▂▂▃▃▂▂▂▂▂▂▂▂▃▃▃▃▃▁▂▂▁▁
train_loss,█▆▆▆▆▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mape,█▆▅▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,0.08104
test_mape,20.49716
time_test,1.4938
time_train,2.81541
train_loss,0.06382


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

largest_num = 6e+7
transformer = models[0]['transform']
model = models[0]['model']
model.eval()
model = model.to('cpu')

df_exclude = df.drop(columns=['price_cleaned'])
df_exclude_np = df_exclude.values
input = transformer(torch.tensor(df_exclude_np, dtype=torch.float32))
output = model(input)
output_cliped = torch.clamp(output, 6.0, 18.0)
output = torch.exp(output_cliped)

plt.scatter(df['price_cleaned'], output.detach().numpy(), s=10, alpha=1)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.ylim([0, largest_num])
plt.xlim([0, largest_num])
plt.plot([0, largest_num], [0, largest_num], color='red')

# Set the axes to display whole numbers
plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

# rotate the x-axis labels
plt.xticks(rotation=45)

plt.show()

In [19]:
# Print model summary
print(models[0]['model'])


FullyConnectedModel(
  (fc1): Linear(in_features=77, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=1, bias=True)
)


AttributeError: 'FullyConnectedModel' object has no attribute 'summary'

In [12]:
import torch

trues = torch.tensor(df['price_cleaned'].values, dtype=torch.float32)


print(f'min outputs: {torch.min(output).item():.4f}, max outputs: {torch.max(output).item():.4f}')
# Berechnung des R^2-Werts
ss_res = torch.sum((trues - output) ** 2)
ss_tot = torch.sum((trues - torch.mean(trues)) ** 2)

print(f"ss_res: {ss_res.item():.4f}, ss_tot: {ss_tot.item():.4f}")

r2_score = 1 - (ss_res / ss_tot)  # R^2-Formel
print(f"R^2-Wert: {r2_score.item():.4f}")

min outputs: 44824.0781, max outputs: 48190656.0000
ss_res: 2525576731257844269056.0000, ss_tot: 61300058171113472.0000
R^2-Wert: -41199.2344


In [5]:
# get index of max prediction to get original data
index = torch.argmax(output)

print(f'index: {index}, max value: {output[index].item():.0f}')
df.iloc[[int(index)]]

index: 366, max value: 65659968


,Floor,detail_responsive#surface_usable,Floor_space_merged,gde_foreigners_percentage,gde_population,gde_social_help_quota,gde_tax,price_cleaned,Space extracted,Plot_area_unified,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
366,0.0,800.0,0.0,28.144885,2043.0,0.689995,5.67,795000.0,200.0,980.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
index_nr = 1
df_exclude = df.drop(columns=['price_cleaned'])
datapoint = df_exclude.iloc[index_nr].values
print(datapoint)
datapoint = torch.tensor(datapoint, dtype=torch.float32).unsqueeze(0)
transformer = models[0]['transform']

datapoint = transformer(datapoint)


model = models[0]['model']
model.eval()
model = model.to('cpu')
output = model(datapoint)
print(f'output: {output}')
transformer = models[0]['transform']
print(f'mean: {transformer.mean}, std: {transformer.std}')
#output = transformer.inverse(output)
#print(f'output inverst: {output}')
output = torch.exp(output)
print(f'output exp: {output[0][0]:.0f}')

[0.00000000e+00 0.00000000e+00 2.42000000e+02 9.25566343e+00
 1.54500000e+03 2.23425863e+00 5.89000000e+00 1.56000000e+02
 2.22000000e+02 5.00000000e+00 0.00000000e+00 0.00000000e+00
 8.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00]
output: tensor([[13.5669]], grad_fn=<AddmmBackward0>)
mean: tensor([2.1500e+00, 8.8294e-01, 2.4629e+01, 2.4678e+01, 1.3220e+04, 2.7662e+00,
        6.2422e+00, 1.5121e+02, 4.6831e+02, 4.8767e+00, 2.1362e+02, 6.3205e+02,
        1.0192e+01, 4.8822e-02, 1.1532e-01, 4.1612e-01, 3.1292e-02, 5.7949e-04,
        2.8974e-04, 3.1485e-02, 2.4184e-01, 1.9316e-04, 3.2500e-02, 7.2919e-03,
        4.8629e-01, 1.1590e-03, 1.8833e-03, 2.9361e-02

In [7]:
df['price_cleaned'].iloc[index_nr]

np.float64(1420000.0)

In [8]:
df.head()

,Floor,detail_responsive#surface_usable,Floor_space_merged,gde_foreigners_percentage,gde_population,gde_social_help_quota,gde_tax,price_cleaned,Space extracted,Plot_area_unified,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,4.000000,121.247562,136.313355,9.255663,1545.0,2.234259,5.89,1150000.0,100.0,1340.042762,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.861332,224.500270,242.000000,9.255663,1545.0,2.234259,5.89,1420000.0,156.0,222.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.000000,129.453708,120.182857,21.358623,21036.0,3.549010,6.05,720000.0,93.0,462.657966,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.126916,232.231722,257.000000,9.255663,1545.0,2.234259,5.89,1430000.0,154.0,370.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,190.906461,177.781407,15.901990,6081.0,1.708126,6.30,995000.0,142.0,32.776514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
train_loader, val_loader, test_loader = torchModelRun.getDataLoaders(df, 'price_cleaned', 1)

counter = 5
for batch in train_loader:
    print(batch)
    counter -= 1
    if counter == 0:
        break

ValueError: too many values to unpack (expected 3)

In [ ]:
df.std().values.reshape(1, -1)

In [ ]:
df.columns

In [ ]:
import numpy as np
input = 1150000.0
log = np.log(input+1)
min = 0.6931471824645996
max = 17.70733070373535
output = (log - min) / (max - min)

print(f'output: {output}')

output = output * (max - min) + min
output = np.exp(output) - 1
print(f'output: {output}')